In [5]:
# Prolog - Auto Generated #
import os, uuid, matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot
import pandas

import sys
sys.tracebacklimit = 0

os.chdir(u'C:/Users/andre/PythonEditorWrapper_3d3105c3-e7de-45f2-9ada-f10985fdc807')
dataset = pandas.read_csv('input_df_087536ac-769a-4aee-b2a8-55626a2dd7e8.csv')

#matplotlib.pyplot.figure(figsize=(5.55555555555556,4.16666666666667), dpi=72)
#matplotlib.pyplot.show = lambda args=None,kw=None: matplotlib.pyplot.savefig(str(uuid.uuid1()))
# Original Script. Please update your script content here and once completed copy below section back to the original editing window #
intro = '''
PROLOG SECTION
======================
uncomment to test in Pyton
'''
# import pandas as pd
# dataset = pd.read_csv("Multichase.csv",index_col=False)
# dataset
begin = '''
BEGIN
=======
copy code below this line'''
# The following code to create a dataframe and remove duplicated rows is always executed and acts as a preamble for your script: 
# dataset = pandas.DataFrame(GFlops, Model)
# dataset = dataset.drop_duplicates()
# Paste or type your script code here:
#dependencies
import pandas as pd
import itertools
from matplotlib.cbook import _reshape_2D
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats.mstats import mquantiles
# Function adapted from matplotlib.cbook
def my_boxplot_stats(X, whis=1.5, bootstrap=None, labels=None,
                  autorange=False, percents=[25, 75]):
    def _bootstrap_median(data, N=5000):
        # determine 95% confidence intervals of the median
        M = len(data)
        percentiles = [2.5, 97.5]
        bs_index = np.random.randint(M, size=(N, M))
        bsData = data[bs_index]
        estimate = np.median(bsData, axis=1, overwrite_input=True)
        CI = np.percentile(estimate, percentiles)
        return CI
    def _compute_conf_interval(data, med, iqr, bootstrap):
        if bootstrap is not None:
            # Do a bootstrap estimate of notch locations.
            # get conf. intervals around median
            CI = _bootstrap_median(data, N=bootstrap)
            notch_min = CI[0]
            notch_max = CI[1]
        else:
            N = len(data)
            notch_min = med - 1.57 * iqr / np.sqrt(N)
            notch_max = med + 1.57 * iqr / np.sqrt(N)
        return notch_min, notch_max
    # output is a list of dicts
    bxpstats = []
    # convert X to a list of lists
    X = _reshape_2D(X, "X")
    ncols = len(X)
    if labels is None:
        labels = itertools.repeat(None)
    elif len(labels) != ncols:
        raise ValueError("Dimensions of labels and X must be compatible")
    input_whis = whis
    for ii, (x, label) in enumerate(zip(X, labels)):
        # empty dict
        stats = {}
        if label is not None:
            stats['label'] = label
        # restore whis to the input values in case it got changed in the loop
        whis = input_whis
        # note tricksyness, append up here and then mutate below
        bxpstats.append(stats)
        # if empty, bail
        if len(x) == 0:
            stats['fliers'] = np.array([])
            stats['mean'] = np.nan
            stats['med'] = np.nan
            stats['q1'] = np.nan
            stats['q3'] = np.nan
            stats['cilo'] = np.nan
            stats['cihi'] = np.nan
            stats['whislo'] = np.nan
            stats['whishi'] = np.nan
            stats['med'] = np.nan
            continue
        # up-convert to an array, just to be safe
        x = np.asarray(x)
        # arithmetic mean
        stats['mean'] = np.mean(x)
        # median
        prob = [0.25,0.5,0.75]
        #med = np.percentile(x, 50)
        ## Altered line
        #q1, q3 = np.percentile(x, (percents[0], percents[1]))
        #change this function to vary quantile profile
        q1, med,q3 = mquantiles(x,prob,alphap=0, betap=0)
        
        # interquartile range
        stats['iqr'] = q3 - q1
        if stats['iqr'] == 0 and autorange:
            whis = 'range'
        # conf. interval around median
        stats['cilo'], stats['cihi'] = _compute_conf_interval(
            x, med, stats['iqr'], bootstrap
        )
        # lowest/highest non-outliers
        if np.isscalar(whis):
            if np.isreal(whis):
                loval = q1 - whis * stats['iqr']
                hival = q3 + whis * stats['iqr']
            elif whis in ['range', 'limit', 'limits', 'min/max']:
                loval = np.min(x)
                hival = np.max(x)
            else:
                raise ValueError('whis must be a float, valid string, or list '
                                 'of percentiles')
        else:
            loval = np.percentile(x, whis[0])
            hival = np.percentile(x, whis[1])
        # get high extreme
        wiskhi = np.compress(x <= hival, x)
        if len(wiskhi) == 0 or np.max(wiskhi) < q3:
            stats['whishi'] = q3
        else:
            stats['whishi'] = np.max(wiskhi)
        # get low extreme
        wisklo = np.compress(x >= loval, x)
        if len(wisklo) == 0 or np.min(wisklo) > q1:
            stats['whislo'] = q1
        else:
            stats['whislo'] = np.min(wisklo)
        # compute a single array of outliers
        stats['fliers'] = np.hstack([
            np.compress(x < stats['whislo'], x),
            np.compress(x > stats['whishi'], x)
        ])
        # add in the remaining stats
        stats['q1'], stats['med'], stats['q3'] = q1, med, q3
    return bxpstats
#used to remove 'nan' from columns when a GUID was supplied
def no_non(lst):
    outp = [x for x in lst if str(x) != 'nan'] 
    return outp
#takes label for column and its data and returns teh calcs formatted for a boxplot tick
def get_legend (data,label):
    
    plot_metric = label+"\n"
    plot_min = "\nmin: "+str(np.round(np.min(data),2))
    plot_mean = "\nmean: "+str(np.round(np.mean(data),2))
    plot_median = "\nmedian: "+str(np.round(np.median(data),2))
    plot_max = "\nmax: "+str(np.round(np.max(data),2))
    plot_qty = "\nN = "+str(len(data))
    plot_cv = "\nCV = "+str(np.round(100.0*np.std(data)/np.mean(data),2))+"%"
    
    return plot_metric + plot_min + plot_max + plot_mean + plot_median + plot_qty + plot_cv
warn = '''
WARNING
========================
Code supports two schemas without editing:
1. a table with col1 as metric and col2 as series with each distinct col2 value making a separate boxplot 
2. a table with only numerics and each column as a separate boxplot
'''
#optional - detect long or wide
if "object" in str(dataset.dtypes):
    
    df = pd.pivot_table(
            dataset
            #change here to standarize on positions
            , values= [dataset.columns[0]]
            #change here to standarize on positions
            , columns=[dataset.columns[1]]
            ,index=dataset.index
            , aggfunc=np.mean)
#extract columns from dataframe into alist of numpy arrays for rendering compatibility
dfx2 = 
    [
        no_non(x) for x in
            [ df.iloc[:,df.columns.get_loc(x)].to_numpy() 
                 for x in df.columns]]
#extract series labels/ticks
cols = [col for col in df.columns]
lbl = [col[1] for col in cols]
#generate list of positions for each boxplot to render
boxlist = {col: my_boxplot_stats(dfx2[cols.index(col)]) for col in cols}
boxout = [boxlist[x][0] for x in boxlist]
#boxplot style
boxprops = {'linewidth': 1, 'color': 'blue'}
flierprops = {'marker': 'o','markerfacecolor': 'green','markersize': 12,'linestyle': 'none'}
medianprops = {'linewidth': 2, 'color': 'blue'}
meanpointprops = {'marker': 'X','linewidth': 1,'markeredgecolor': 'blue','markerfacecolor': 'blue'}
# Plot boxplots from our computed statistics
fig, ax = plt.subplots(1, 1,figsize=(int(len(dfx2)*2),
#I had to adjust this number to get all the legend to show up
    14 ))
boxpos = len(boxout)+1
Q = ax.bxp(
    boxout
    , widths = 0.75
    , positions=range(len(boxout))
    , showmeans = True
    , boxprops=boxprops
    , medianprops = medianprops
    , whiskerprops = boxprops
    , capprops = boxprops 
    , meanprops = meanpointprops
)
#legend calcs
lgnd = [get_legend(dfx2[i],lbl[i]) for i in range(len(dfx2))]
ax.grid(None)
ax.spines['top'].set_color('#FFFFFF')
ax.spines['right'].set_color('#FFFFFF')
ax.spines['bottom'].set_color('#FFFFFF')
ax.spines['left'].set_color('#FFFFFF')
#EDIT HERE FOR CUSTOM TITLE
#ax.set_title('GFlops')
ax.set_title(dataset.columns[0])
plt.xticks([x for x in range(len(dfx2))], lgnd)
#jitter
color = 'tab:red'
jitterx =[(i)+(0.5*(np.random.random() - 0.5))  for i in range(len(dfx2)) for x in dfx2[i]]
jittery = [x[z] for  x in dfx2 for z in range(len(x))]
ax.scatter(jitterx, jittery, color=color)
#render
plt.show()

# Epilog - Auto Generated #
os.chdir(u'C:/Users/andre/PythonEditorWrapper_3d3105c3-e7de-45f2-9ada-f10985fdc807')


In [14]:
dfx2

[[1261864.908, 1692332.514, 1971830.6840000001],
 [1006956.441,
  1037162.421,
  1053511.892,
  1075731.414,
  1102698.18,
  1177175.027,
  1211177.592,
  1214469.459,
  1451462.13,
  1640483.9409999999,
  1663659.013,
  1674989.6430000002,
  1790222.2119999998,
  1847138.185,
  1848205.5659999999,
  1874962.7,
  1932392.654,
  2026157.9780000001,
  2038852.7680000002,
  2045968.9209999999,
  2084579.1819999998,
  2093127.289,
  2101696.192,
  2131745.9,
  2137622.1,
  2249636.558,
  2259979.503,
  2268574.647,
  2269322.4850000003,
  2338321.134,
  2410220.147,
  2861265.16,
  2992295.262,
  3115544.915,
  3379652.04,
  3537195.376,
  3870669.785,
  4035491.986,
  4744903.795,
  5480969.397999999,
  6748815.072000001,
  7086463.129,
  7501392.913,
  8414895.002999999,
  9039830.197999999,
  9172687.257000001,
  9741319.553,
  11922504.742,
  14789784.318,
  18710444.067,
  28234807.283000004],
 [1056327.479,
  1116114.017,
  1161573.607,
  1206201.301,
  1216727.01,
  1237837.13,
  13

In [7]:
boxlist

{('Amount',
  'Caribbean and Central America'): [{'mean': 1642009.3686666668,
   'iqr': 709965.7760000001,
   'cilo': 1048791.190934236,
   'cihi': 2335873.837065764,
   'whishi': 1971830.6840000001,
   'whislo': 1261864.908,
   'fliers': array([], dtype=float64),
   'q1': 1261864.908,
   'med': 1692332.514,
   'q3': 1971830.6840000001}],
 ('Amount',
  'Central and Northern Asia'): [{'mean': 4322335.971098038,
   'iqr': 2954681.583,
   'cilo': 1600067.6192748775,
   'cihi': 2899205.496725123,
   'whishi': 9172687.257000001,
   'whislo': 1006956.441,
   'fliers': array([ 9741319.553, 11922504.742, 14789784.318, 18710444.067,
          28234807.283]),
   'q1': 1790222.2119999998,
   'med': 2249636.558,
   'q3': 4744903.795}],
 ('Amount',
  'China'): [{'mean': 5446547.090103448,
   'iqr': 4762684.978999998,
   'cilo': 1977745.9514443136,
   'cihi': 4754788.320555687,
   'whishi': 12708433.14,
   'whislo': 1056327.479,
   'fliers': array([15315664.853, 29824116.889]),
   'q1': 1497723.742,

In [8]:
df

Amount                                  \
Region Caribbean and Central America Central and Northern Asia China   
0                                NaN                       NaN   NaN   
1                                NaN                       NaN   NaN   
2                                NaN               1006956.441   NaN   
3                                NaN                       NaN   NaN   
4                                NaN                       NaN   NaN   
..                               ...                       ...   ...   
534                              NaN                       NaN   NaN   
535                              NaN                       NaN   NaN   
536                              NaN                       NaN   NaN   
537                              NaN                       NaN   NaN   
538                              NaN                       NaN   NaN   

                                                                      \
Region East Asia excluding China Europe Middle East and North Africa   
0                            NaN    NaN                          NaN   
1                    1004998.208    NaN                          NaN   
2                            NaN    NaN                          NaN   
3                            NaN    NaN                  1007324.631   
4                            NaN    NaN                          NaN   
..                           ...    ...                          ...   
534                          NaN    NaN                          NaN   
535                          NaN    NaN                          NaN   
536                          NaN    NaN                          NaN   
537                          NaN    NaN                          NaN   
538                          NaN    NaN                          NaN   

                                                                            \
Region North America       Oceania South America South Asia Southeast Asia   
0                NaN           NaN           NaN        NaN            NaN   
1                NaN           NaN           NaN        NaN            NaN   
2                NaN           NaN           NaN        NaN            NaN   
3                NaN           NaN           NaN        NaN            NaN   
4                NaN           NaN           NaN        NaN    1009311.464   
..               ...           ...           ...        ...            ...   
534              NaN           NaN  3.924335e+07        NaN            NaN   
535     4.183752e+07           NaN           NaN        NaN            NaN   
536     4.570303e+07           NaN           NaN        NaN            NaN   
537              NaN  6.446875e+07           NaN        NaN            NaN   
538     1.057812e+08           NaN           NaN        NaN            NaN   

                           
Region Sub-Saharan Africa  
0             1000183.856  
1                     NaN  
2                     NaN  
3                     NaN  
4                     NaN  
..                    ...  
534                   NaN  
535                   NaN  
536                   NaN  
537                   NaN  
538                   NaN  

[539 rows x 12 columns]

In [9]:
dfx2

[[1261864.908, 1692332.514, 1971830.6840000001],
 [1006956.441,
  1037162.421,
  1053511.892,
  1075731.414,
  1102698.18,
  1177175.027,
  1211177.592,
  1214469.459,
  1451462.13,
  1640483.9409999999,
  1663659.013,
  1674989.6430000002,
  1790222.2119999998,
  1847138.185,
  1848205.5659999999,
  1874962.7,
  1932392.654,
  2026157.9780000001,
  2038852.7680000002,
  2045968.9209999999,
  2084579.1819999998,
  2093127.289,
  2101696.192,
  2131745.9,
  2137622.1,
  2249636.558,
  2259979.503,
  2268574.647,
  2269322.4850000003,
  2338321.134,
  2410220.147,
  2861265.16,
  2992295.262,
  3115544.915,
  3379652.04,
  3537195.376,
  3870669.785,
  4035491.986,
  4744903.795,
  5480969.397999999,
  6748815.072000001,
  7086463.129,
  7501392.913,
  8414895.002999999,
  9039830.197999999,
  9172687.257000001,
  9741319.553,
  11922504.742,
  14789784.318,
  18710444.067,
  28234807.283000004],
 [1056327.479,
  1116114.017,
  1161573.607,
  1206201.301,
  1216727.01,
  1237837.13,
  13

In [10]:
boxout

[{'mean': 1642009.3686666668,
  'iqr': 709965.7760000001,
  'cilo': 1048791.190934236,
  'cihi': 2335873.837065764,
  'whishi': 1971830.6840000001,
  'whislo': 1261864.908,
  'fliers': array([], dtype=float64),
  'q1': 1261864.908,
  'med': 1692332.514,
  'q3': 1971830.6840000001},
 {'mean': 4322335.971098038,
  'iqr': 2954681.583,
  'cilo': 1600067.6192748775,
  'cihi': 2899205.496725123,
  'whishi': 9172687.257000001,
  'whislo': 1006956.441,
  'fliers': array([ 9741319.553, 11922504.742, 14789784.318, 18710444.067,
         28234807.283]),
  'q1': 1790222.2119999998,
  'med': 2249636.558,
  'q3': 4744903.795},
 {'mean': 5446547.090103448,
  'iqr': 4762684.978999998,
  'cilo': 1977745.9514443136,
  'cihi': 4754788.320555687,
  'whishi': 12708433.14,
  'whislo': 1056327.479,
  'fliers': array([15315664.853, 29824116.889]),
  'q1': 1497723.742,
  'med': 3366267.1360000004,
  'q3': 6260408.720999999},
 {'mean': 3370006.800576923,
  'iqr': 2153191.4877500003,
  'cilo': 1567910.6038197526

In [11]:
ax.bxp?

In [12]:
lgnd

['Caribbean and Central America\n\nmin: 1261864.91\nmax: 1971830.68\nmean: 1642009.37\nmedian: 1692332.51\nN = 3\nCV = 17.78%',
 'Central and Northern Asia\n\nmin: 1006956.44\nmax: 28234807.28\nmean: 4322335.97\nmedian: 2249636.56\nN = 51\nCV = 115.79%',
 'China\n\nmin: 1056327.48\nmax: 29824116.89\nmean: 5446547.09\nmedian: 3366267.14\nN = 29\nCV = 109.64%',
 'East Asia excluding China\n\nmin: 1004998.21\nmax: 12759040.05\nmean: 3370006.8\nmedian: 2230883.28\nN = 26\nCV = 91.01%',
 'Europe\n\nmin: 1019810.49\nmax: 32877595.64\nmean: 4675830.66\nmedian: 2803294.09\nN = 123\nCV = 105.25%',
 'Middle East and North Africa\n\nmin: 1007324.63\nmax: 18859279.35\nmean: 4429239.51\nmedian: 2656466.42\nN = 88\nCV = 98.11%',
 'North America\n\nmin: 1009328.72\nmax: 105781220.6\nmean: 8831869.47\nmedian: 2215474.73\nN = 47\nCV = 201.42%',
 'Oceania\n\nmin: 1066416.61\nmax: 64468746.33\nmean: 8992974.58\nmedian: 2552927.58\nN = 16\nCV = 176.06%',
 'South America\n\nmin: 1010845.21\nmax: 39243347.0